In [1]:
%config ZMQInteractiveShell.ast_node_interactivity='all'
import pandas as pd
from config import *


from gensim.models import Word2Vec
import multiprocessing
import jieba
from collections import Counter
import json
import numpy as np

## 加载源数据

In [2]:
reviews = []
labels = []
with open(dataSource,'r',encoding='utf-8') as file:
    for line in file:
        temp = line.replace('\n', '').split(',,')
        reviews.append(temp[0])
        labels.append(temp[1])
print('data:',len(reviews),len(labels))
reviews[:2]
labels[:2]

data: 101058 101058


['才用就发现相机打开迟钝，半天反应不过来，有时候还会卡出去，他们又不给解决方案。', '还没穿二天就起毛了']

['0', '0']

In [3]:
reviews = [jieba.lcut(review.replace('\n', '')) for review in reviews]
print('data:',len(reviews),len(labels))
reviews[:2]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\zhoubin\AppData\Local\Temp\jieba.cache
Loading model cost 0.975 seconds.
Prefix dict has been built succesfully.


data: 101058 101058


[['才',
  '用',
  '就',
  '发现',
  '相机',
  '打开',
  '迟钝',
  '，',
  '半天',
  '反应',
  '不',
  '过来',
  '，',
  '有时候',
  '还会卡',
  '出去',
  '，',
  '他们',
  '又',
  '不',
  '给',
  '解决方案',
  '。'],
 ['还', '没', '穿', '二天', '就', '起毛', '了']]

## 预训练词向量

In [4]:
# w2v_model = Word2Vec(reviews,size=embeddingSize,
#                      min_count=miniFreq,
#                      window=10,
#                      workers=multiprocessing.cpu_count(),sg=1,
#                      iter=20)

# w2v_model = Word2Vec(reviews,size=embeddingSize,
#                      min_count=miniFreq,
#                      window=10,
#                      workers=10,sg=1,
#                      iter=20)
# w2v_model.save(w2v_model_path)

# model = Word2Vec.load(w2v_model_path)
# model.wv.vocab.keys()

## 加载停用词

In [5]:
def readStopWord(stopWordPath):
    stopWordDict = {}
    with open(stopWordPath, "r", encoding='utf-8') as f:
        stopWords = f.read()
        stopWordList = stopWords.splitlines()
        # 将停用词用列表的形式生成，之后查找停用词时会比较快
        stopWordDict = dict(zip(stopWordList, list(range(len(stopWordList)))))
    return stopWordDict

stopWordDict = readStopWord(stopWordPath)
# stopWordDict

## 构建词典

In [6]:
def genVocabulary():
    """
    按照我们的数据集中的单词取出预训练好的word2vec中的词向量
    """

    # 中文
    model = Word2Vec.load(w2v_model_path)

    vocab = []
    wordEmbedding = [] # 预训练权重矩阵

    # 添加 "pad" 和 "UNK", 
    vocab.append("pad")
    wordEmbedding.append(np.zeros(embeddingSize))

    vocab.append("UNK")
    wordEmbedding.append(np.random.randn(embeddingSize))
    
        
    allWords = [word for review in reviews for word in review]
    # 去掉停用词
    subWords = [word for word in allWords if word not in stopWordDict]
    # 统计词频，排序
    wordCount = Counter(subWords)
    sortWordCount = sorted(wordCount.items(), key=lambda x: x[1], reverse=True)
    # 去除低频词
    words = [item[0] for item in sortWordCount if item[1] >= miniFreq]
    
    # 获取词列表和顺序对应的预训练权重矩阵
    for word in words:
        try:
            vector = model[word]
            vocab.append(word)
            wordEmbedding.append(vector)

        except:
            print(word + "不存在于词向量中")
            
    
    wordToIndex = dict(zip(vocab, list(range(len(vocab)))))
    indexToWord = dict(zip(list(range(len(vocab))), vocab))
    
    # 将词汇-索引映射表保存为json数据，之后做inference时直接加载来处理数据
    with open("./temp/wordJson/wordToIndex.json", "w", encoding="utf-8") as f:
        json.dump(wordToIndex, f)

    with open("./temp/wordJson/indexToWord.json", "w", encoding="utf-8") as f:
        json.dump(indexToWord, f)

    return vocab, np.array(wordEmbedding), wordToIndex, indexToWord


vocab, wordEmbedding, wordToIndex, indexToWord = genVocabulary()

E:\Anaconda\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
E:\Anaconda\lib\site-packages\ipykernel_launcher.py:32: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


## 构建数据集

In [7]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

def reviewProcess(review, sequenceLength, wordToIndex):
    """
    将数据集中的每条评论里面的词，根据词表，映射为index表示
    每条评论 用index组成的定长数组来表示

    """

    reviewVec = np.zeros((sequenceLength))
    
    sequenceLen = sequenceLength

    # 判断当前的序列是否小于定义的固定序列长度
    if len(review) < sequenceLength:
        sequenceLen = len(review)

    for i in range(sequenceLen):
        if review[i] in wordToIndex:
            reviewVec[i] = wordToIndex[review[i]]
        else:
            reviewVec[i] = wordToIndex["UNK"]

    return reviewVec



def genTrainEvalData(x, y, rate):
    """
    生成训练集和验证集
    """

    reviews = []
    labels = []

    # 遍历所有的文本，将文本中的词转换成index表示
    for i in range(len(x)):
        reviewVec = reviewProcess(x[i], sequenceLength, wordToIndex)
        reviews.append(reviewVec)
        labels.append([y[i]])

    trainIndex = int(len(x) * rate)

#     trainReviews = pad_sequences(reviews[:trainIndex], maxlen=sequenceLength)
    trainReviews = np.asarray(reviews[:trainIndex], dtype="int64")
    trainLabels = np.array(labels[:trainIndex], dtype="float32")
    
    trainLabels = to_categorical(trainLabels,num_classes=2)

#     evalReviews = pad_sequences(reviews[trainIndex:], maxlen=sequenceLength)
    evalReviews = np.asarray(reviews[trainIndex:], dtype="int64")
    evalLabels = np.array(labels[trainIndex:], dtype="float32")
    
    evalLabels = to_categorical(evalLabels,num_classes=2)

    return trainReviews, trainLabels, evalReviews, evalLabels


trainReviews, trainLabels, evalReviews, evalLabels = genTrainEvalData(reviews, labels, rate=rate)
print("train data shape: {}".format(trainReviews.shape))
print("train label shape: {}".format(trainLabels.shape))
print("eval data shape: {}".format(evalReviews.shape))
print("eval label shape: {}".format(evalLabels.shape))
trainReviews[:2]
trainLabels[:2]

train data shape: (80846, 200)
train label shape: (80846, 2)
eval data shape: (20212, 200)
eval label shape: (20212, 2)


array([[   1,    1,    1,   71,  340,   79, 1923,    1,  274,    1,    1,
           1,    1,  260, 6621,    1,    1,    1,    1,    1,    1, 2039,
           1,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 

array([[1., 0.],
       [1., 0.]], dtype=float32)

## 模型构建

In [8]:
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D,GlobalAveragePooling1D,concatenate,Flatten,Dense,Dropout,Embedding,Reshape,LSTM,Layer,Bidirectional,GRU,BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras import Sequential,optimizers,losses
from tensorflow.keras import backend
backend.clear_session() 
from tensorflow.keras import backend as K
K.clear_session()


class Position_Embedding(Layer):
    def __init__(self, size=None, mode='sum', **kwargs):
        self.size = size #必须为偶数
        self.mode = mode
        super(Position_Embedding, self).__init__(**kwargs)

    def call(self, x):
        if (self.size == None) or (self.mode == 'sum'):
            self.size = int(x.shape[-1])
        batch_size,seq_len = K.shape(x)[0],K.shape(x)[1]
        position_j = 1. / K.pow(10000., 2 * K.arange(self.size / 2, dtype='float32') / self.size)
        position_j = K.expand_dims(position_j, 0)
        position_i = K.cumsum(K.ones_like(x[:,:,0]), 1)-1 #K.arange不支持变长，只好用这种方法生成
        position_i = K.expand_dims(position_i, 2)
        position_ij = K.dot(position_i, position_j)
        position_ij = K.concatenate([K.cos(position_ij), K.sin(position_ij)], 2)
        if self.mode == 'sum':
            return position_ij + x
        elif self.mode == 'concat':
            return K.concatenate([position_ij, x], 2)

    def compute_output_shape(self, input_shape):
        if self.mode == 'sum':
            return input_shape
        elif self.mode == 'concat':
            return (input_shape[0], input_shape[1], input_shape[2]+self.size)


class Attention(Layer):
    def __init__(self, nb_head, size_per_head, **kwargs):
        self.nb_head = nb_head
        self.size_per_head = size_per_head
        self.output_dim = nb_head*size_per_head
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.WQ = self.add_weight(name='WQ',
                                  shape=(input_shape[0][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WK = self.add_weight(name='WK',
                                  shape=(input_shape[1][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WV = self.add_weight(name='WV',
                                  shape=(input_shape[2][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        super(Attention, self).build(input_shape)

    def Mask(self, inputs, seq_len, mode='mul'):
        if seq_len == None:
            return inputs
        else:
            mask = K.one_hot(seq_len[:,0], K.shape(inputs)[1])
            mask = 1 - K.cumsum(mask, 1)
            for _ in range(len(inputs.shape)-2):
                mask = K.expand_dims(mask, 2)
            if mode == 'mul':
                return inputs * mask
            if mode == 'add':
                return inputs - (1 - mask) * 1e12

    def call(self, x):
        #如果只传入Q_seq,K_seq,V_seq，那么就不做Mask
        #如果同时传入Q_seq,K_seq,V_seq,Q_len,V_len，那么对多余部分做Mask
        if len(x) == 3:
            Q_seq,K_seq,V_seq = x
            Q_len,V_len = None,None
        elif len(x) == 5:
            Q_seq,K_seq,V_seq,Q_len,V_len = x
        #对Q、K、V做线性变换
        Q_seq = K.dot(Q_seq, self.WQ)
        Q_seq = K.reshape(Q_seq, (-1, K.shape(Q_seq)[1], self.nb_head, self.size_per_head))
        Q_seq = K.permute_dimensions(Q_seq, (0,2,1,3))
        K_seq = K.dot(K_seq, self.WK)
        K_seq = K.reshape(K_seq, (-1, K.shape(K_seq)[1], self.nb_head, self.size_per_head))
        K_seq = K.permute_dimensions(K_seq, (0,2,1,3))
        V_seq = K.dot(V_seq, self.WV)
        V_seq = K.reshape(V_seq, (-1, K.shape(V_seq)[1], self.nb_head, self.size_per_head))
        V_seq = K.permute_dimensions(V_seq, (0,2,1,3))
        #计算内积，然后mask，然后softmax
        A = K.batch_dot(Q_seq, K_seq, axes=[3,3]) / self.size_per_head**0.5
        A = K.permute_dimensions(A, (0,3,2,1))
        A = self.Mask(A, V_len, 'add')
        A = K.permute_dimensions(A, (0,3,2,1))
        A = K.softmax(A)
        #输出并mask
        O_seq = K.batch_dot(A, V_seq, axes=[3,2])
        O_seq = K.permute_dimensions(O_seq, (0,2,1,3))
        O_seq = K.reshape(O_seq, (-1, K.shape(O_seq)[1], self.output_dim))
        O_seq = self.Mask(O_seq, Q_len, 'mul')
        return O_seq

    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][1], self.output_dim)
    
    
    
def transfromer(max_features, embedding_weights, maxlen):
    S_inputs = Input(shape=(None,), dtype='int32')

    embeddings = Embedding(input_dim=max_features, output_dim=embeddingSize,
                            weights=[embedding_weights],
                            input_length=maxlen)(S_inputs)

    #增加Position_Embedding能轻微提高准确率
    embeddings = Position_Embedding()(embeddings) 

    O_seq = Attention(8,16)([embeddings,embeddings,embeddings])

    O_seq = GlobalAveragePooling1D()(O_seq)

    O_seq = Dropout(dropoutKeepProb)(O_seq)

    outputs = Dense(2, activation='softmax')(O_seq)

    model = Model(inputs=S_inputs, outputs=outputs)
    
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

## 模型训练

In [9]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
import yaml

x_train = trainReviews
y_train = trainLabels
x_eval = evalReviews
y_eval = evalLabels

max_features = len(wordToIndex) + 1

print('构建模型...')
model = transfromer(max_features, wordEmbedding, sequenceLength)
model.summary()

# 回调函数
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=10, mode='auto')
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model_checkpoint = ModelCheckpoint('./model/transform_model/model_{epoch:02d}-{val_accuracy:.2f}.hdf5',
                                   save_best_only=True, save_weights_only=True)


history = model.fit(x_train, y_train, batch_size=batchSize, epochs=epochs, validation_split=0.3,
                    shuffle=True, callbacks=[reduce_lr,early_stopping,model_checkpoint])
#验证
scores = model.evaluate(x_eval, y_eval)

#保存模型
yaml_string = model.to_yaml()
with open('./model/transfromer.yml', 'w') as outfile:
    outfile.write( yaml.dump(yaml_string, default_flow_style=True))
model.save_weights('./model/transfromer.h5')

print('test_loss: %f, accuracy: %f' % (scores[0], scores[1]))

构建模型...


ValueError: in converted code:

    <ipython-input-8-8c9619d5189e>:94 call  *
        A = K.permute_dimensions(A, (0,3,2,1))
    E:\Anaconda\lib\site-packages\tensorflow_core\python\keras\backend.py:2768 permute_dimensions
        return array_ops.transpose(x, perm=pattern)
    E:\Anaconda\lib\site-packages\tensorflow_core\python\ops\array_ops.py:1870 transpose
        ret = transpose_fn(a, perm, name=name)
    E:\Anaconda\lib\site-packages\tensorflow_core\python\ops\gen_array_ops.py:11454 transpose
        "Transpose", x=x, perm=perm, name=name)
    E:\Anaconda\lib\site-packages\tensorflow_core\python\framework\op_def_library.py:793 _apply_op_helper
        op_def=op_def)
    E:\Anaconda\lib\site-packages\tensorflow_core\python\framework\func_graph.py:548 create_op
        compute_device)
    E:\Anaconda\lib\site-packages\tensorflow_core\python\framework\ops.py:3429 _create_op_internal
        op_def=op_def)
    E:\Anaconda\lib\site-packages\tensorflow_core\python\framework\ops.py:177

## 可视化

In [ ]:
history_dict = history.history
history_dict.keys()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.clf()   # clear figure
acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()